# Spaceship Titanic - fifth Model

## Overview

The Spaceship Titanic was an interstellar passenger liner launched a month ago. With almost 13,000 passengers on board, 
the vessel set out on its maiden voyage transporting emigrants from our solar system to three newly habitable exoplanets 
orbiting nearby stars.

While rounding Alpha Centauri en route to its first destination—the torrid 55 Cancri E—the unwary Spaceship Titanic
collided with a spacetime anomaly hidden within a dust cloud. Sadly, it met a similar fate as its namesake from 1000
years before. Though the ship stayed intact, almost half of the passengers were transported to an alternate dimension!

In this competition your task is to predict whether a passenger was transported to an alternate dimension during the 
Spaceship Titanic's collision with the spacetime anomaly. To help you make these predictions, you're given a set of personal
records recovered from the ship's damaged computer system.

## File and Data Field Descriptions

### train.csv 

Personal records for about two-thirds (~8700) of the passengers, to be used as training data.

Sure! Here's the code converted to a markdown table:

| Column Name | Description |
|------------- |-------------|
| `PassengerId` | A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always. |
| `HomePlanet` | The planet the passenger departed from, typically their planet of permanent residence. |
| `CryoSleep` | Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins. |
| `Cabin` | The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard. |
| `Destination` | The planet the passenger will be debarking to. |
| `Age` | The age of the passenger. |
| `VIP` | Whether the passenger has paid for special VIP service during the voyage. |
| `RoomService`, `FoodCourt`, `ShoppingMall`, `Spa`, `VRDeck` | Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities. |
| `Name` | The first and last names of the passenger. |
| `Transported` | Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict. |

### test.csv

Personal records for the remaining one-third (~4300) of the passengers, to be used as test data. 

Your task is to predict the value of Transported for the passengers in this set.
    
### sample_submission.csv

A sample submission file in the correct format.

| Column Name | Description |
|------------- |-------------|
| `PassengerId` | A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always. |
| `Transported` | Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict. |

In [1]:
model='fifth'

In [2]:
# Library imports

# Data wrangling
import pandas as pd
import numpy as np
import missingno
from collections import Counter

# Data visualisation
import seaborn as sns
import matplotlib.pyplot as plt

# Preprocessing
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Machine learning models
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

# Model evaluation
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

# Hyperparameter tuning
from sklearn.model_selection import GridSearchCV

# Remove warnings
import warnings
warnings.filterwarnings('ignore')

# Data preprocessing

## Drop unrequired features & add engineered ones

In [3]:
# Load the data - to pandas dataframes

test_df = pd.read_csv('./inputs/test.csv')
test_idx = test_df['PassengerId']
train_df = pd.read_csv('./inputs/train.csv')

In [4]:
# split name into components
def split_name(name:str):
    if pd.isnull(name): # protect against empty values
        return (np.nan, np.nan, np.nan)
    parts = name.split(' ')
    first = parts[0]
    last = parts[-1]
    return (first, last)

# split cabin into components
def split_cabin(name:str):
    if pd.isnull(name): # protect against empty values
        return (np.nan, np.nan, np.nan)
    parts = name.split('/')
    deck = parts[0]
    side = parts[-1]
    number = ' '.join(parts[1:-1])
    return (deck, number, side)

# This function will drop and add required features
def feature_manipulation(d : pd.DataFrame) -> pd.DataFrame:
    # print the shape of the provided dataframe
    print("Before: ", d.shape)
    # Clone the provided dataframe
    df = d.copy()

    _, df['Name'] = zip(*df['Name'].map(split_name))
    
    # Location
    df['Deck'], _, df['Side'] = zip(*df['Cabin'].map(split_cabin))
    df['DeckSide'] = df['Deck'] + df['Side'] # Combine Deck and Side

    # Group related columns
    df['Group'] = df['PassengerId'].map(lambda x: x.split('_')[0])
    df['GroupSize'] = df['Group'].map(df['Group'].value_counts())
    
    print("After: ", df.shape)
    return df

In [5]:
train_df = feature_manipulation(train_df)

Before:  (8693, 14)
After:  (8693, 19)


In [6]:
test_df = feature_manipulation(test_df)

Before:  (4277, 13)
After:  (4277, 18)


In [7]:
train_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Side,DeckSide,Group,GroupSize
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Ofracculy,False,B,P,BP,0001,1
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Vines,True,F,S,FS,0002,1
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Susent,False,A,S,AS,0003,2
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Susent,False,A,S,AS,0003,2
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Santantines,True,F,S,FS,0004,1


In [8]:
test_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Deck,Side,DeckSide,Group,GroupSize
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Carsoning,G,S,GS,0013,1
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Peckers,F,S,FS,0018,1
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Unhearfus,C,S,CS,0019,1
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Caltilter,C,S,CS,0021,1
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Harperez,F,S,FS,0023,1


## Handle Missing Data

In the first model we had a complicated approach - here we will just use the median for numeric and mode for non-numeric

In [9]:
print(f'Before: {len(train_df)} rows')
train_df.isnull().sum().sort_values(ascending = False)

Before: 8693 rows


CryoSleep       217
ShoppingMall    208
VIP             203
HomePlanet      201
Name            200
DeckSide        199
Cabin           199
Deck            199
Side            199
VRDeck          188
FoodCourt       183
Spa             183
Destination     182
RoomService     181
Age             179
Group             0
PassengerId       0
Transported       0
GroupSize         0
dtype: int64

In [10]:
print(f'Before: {len(test_df)} rows')
train_df.isnull().sum().sort_values(ascending = False)

Before: 4277 rows


CryoSleep       217
ShoppingMall    208
VIP             203
HomePlanet      201
Name            200
DeckSide        199
Cabin           199
Deck            199
Side            199
VRDeck          188
FoodCourt       183
Spa             183
Destination     182
RoomService     181
Age             179
Group             0
PassengerId       0
Transported       0
GroupSize         0
dtype: int64

In [11]:
# rows with no missing values
print(f'Before: {len(train_df)} rows')
train_nona = train_df.dropna()
print(f'After: {len(train_nona)} rows')

Before: 8693 rows
After: 6606 rows


## Apply the empty data handling

### Define functions to replace data

In [12]:
def fill_with_median(df, features):
    for feature in features:
        df[feature] = df[feature].fillna(df[feature].median())
    return df

def fill_with_mode(df, features):
    for feature in features:
        df[feature] = df[feature].fillna(df[feature].mode()[0])
    return df

def fill_with_mean(df, features):
    for feature in features:
        df[feature] = df[feature].fillna(df[feature].mean())
    return df

# Here we provide a list of other_features - if a row has a nan in a feature we will fill it with the mean of the other_features in its row 
def fill_with_mean_of_other_features(df, features, other_features):
    for feature in features:
        df[feature] = df[feature].fillna(df[other_features].dropna().mean(axis=1))
    return df

def fill_with_constant(df, features, constant):
    for feature in features:
        df[feature] = df[feature].fillna(constant)
    return df

### Apply simple rules for empty data

In [13]:
def handle_empty_data(df: pd.DataFrame) -> pd.DataFrame:
   
    # fill in missing values with a constant
    df = fill_with_constant(df, ['VIP'], False)
    df = fill_with_constant(df, ['Name', 'Cabin', 'Deck', 'Side', 'DeckSide', 'HomePlanet', 'Destination'], 'Unknown')

    # Fill numerics with median
    df = fill_with_median(df, ['Age'])
    
    # Fill with average of other spend features
    spend_features = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    df = fill_with_mean_of_other_features(df, spend_features, spend_features)
    df = fill_with_constant(df, spend_features, 0)

    # Have a sensible default so leave this as is
    # for the provided dataframe set CryoSleep to False if TotalSpend is greater than 0
    UnknownCryoSpender = (df["CryoSleep"].isnull() | df["CryoSleep"].isna()) & (df[spend_features].dropna().sum(axis=1) > 0)
    df.loc[UnknownCryoSpender, 'CryoSleep'] = False
    df['CryoSleep'] = df['CryoSleep'].fillna('Unknown')

    return df


In [14]:
train_df = handle_empty_data(train_df)
test_df = handle_empty_data(test_df)

In [15]:
# Check the training data

train_df.isnull().sum().sort_values(ascending = False)


PassengerId     0
Spa             0
Group           0
DeckSide        0
Side            0
Deck            0
Transported     0
Name            0
VRDeck          0
ShoppingMall    0
HomePlanet      0
FoodCourt       0
RoomService     0
VIP             0
Age             0
Destination     0
Cabin           0
CryoSleep       0
GroupSize       0
dtype: int64

In [16]:
# Check the test data
test_df.isnull().sum().sort_values(ascending = False)

PassengerId     0
HomePlanet      0
Group           0
DeckSide        0
Side            0
Deck            0
Name            0
VRDeck          0
Spa             0
ShoppingMall    0
FoodCourt       0
RoomService     0
VIP             0
Age             0
Destination     0
Cabin           0
CryoSleep       0
GroupSize       0
dtype: int64

### And thats a wrap!

There are now no missing values so we can proceed to Pre-processing

# Data Preprocessing

## Quick check of the data after the empty data is filled

In [17]:
# Look at the training data after the changes to remove empty data
train_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Side,DeckSide,Group,GroupSize
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Ofracculy,False,B,P,BP,0001,1
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Vines,True,F,S,FS,0002,1
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Susent,False,A,S,AS,0003,2
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Susent,False,A,S,AS,0003,2
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Santantines,True,F,S,FS,0004,1


In [18]:
# Look at the test data
test_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Deck,Side,DeckSide,Group,GroupSize
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Carsoning,G,S,GS,0013,1
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Peckers,F,S,FS,0018,1
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Unhearfus,C,S,CS,0019,1
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Caltilter,C,S,CS,0021,1
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Harperez,F,S,FS,0023,1


In [19]:
# Check the shapes:

print(f'Train: {train_df.shape}')
print(f'Test: {test_df.shape}')

# Check the train and test column data types side by side
pd.concat([train_df.dtypes, test_df.dtypes], axis=1, keys=['Train', 'Test'])



Train: (8693, 19)
Test: (4277, 18)


,Train,Test
PassengerId,object,object
HomePlanet,object,object
CryoSleep,object,object
Cabin,object,object
Destination,object,object
Age,float64,float64
VIP,bool,bool
RoomService,float64,float64
FoodCourt,float64,float64
ShoppingMall,float64,float64


## Drop Unrequired columns

We can now drop these columns:

- Cabin
- TotalSpend
- PassengerId - may do this one last so we have the key

And to simplify for now lets also remove
- Name
- Group

In [20]:
# # Drop the columns that are not required
# def drop_unrequired_columns(df: pd.DataFrame) -> pd.DataFrame:
#     columns_to_drop = [
#         'Cabin',
#         'Name',
#         # 'DeckSide'
#         'GroupSize'
#     ]
#     df.drop(columns_to_drop, axis = 1, inplace = True)
#     return df

# train_df = drop_unrequired_columns(train_df)
# test_df = drop_unrequired_columns(test_df)

cols_to_keep = ['CryoSleep',
 'DeckSide',
 'Age',
 'RoomService',
 'Destination',
 'Deck',
 'HomePlanet',
 'VRDeck',
 'Spa']
cols_to_keep.append('PassengerId')
cols_to_keep.append('Transported')

train_df = train_df[cols_to_keep]
test_df = test_df[cols_to_keep[:-1]]


## Prepare the data for training

- Log/Bin the numeric data
- OneHotEncoding

### Bin and OHE features

I am going to try the `KBinsDiscretizer` from sklearn - see https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.KBinsDiscretizer.html#sklearn.preprocessing.KBinsDiscretizer

In [21]:
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import RobustScaler

# Define pipelines for the features where we wish to bin and OHE

spending_pipeline = Pipeline([
    ('scale', RobustScaler()),
])

age_pipeline = Pipeline([
    ('binning', KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')),
    ('onehot', OneHotEncoder())
])

categorical_pipeline = Pipeline([
    ('onehot', OneHotEncoder())
])


In [22]:
# There are some cases where we have used 'Unknown' in boolean columns that have True and False otherwise that led to an error
# complaining about a mix of str and bool so we need to convert the columns to str

# print all columns where the value is 'Unknown'
for column in train_df.columns:
    if 'Unknown' in train_df[column].unique():
        print(column)
        train_df[column] = train_df[column].astype(str)



CryoSleep
DeckSide
Destination
Deck
HomePlanet


## Run the pipeline

In [23]:
# what data looks like before pipeline
train_df.head()

,CryoSleep,DeckSide,Age,RoomService,Destination,Deck,HomePlanet,VRDeck,Spa,PassengerId,Transported
0,False,BP,39.0,0.0,TRAPPIST-1e,B,Europa,0.0,0.0,0001_01,False
1,False,FS,24.0,109.0,TRAPPIST-1e,F,Earth,44.0,549.0,0002_01,True
2,False,AS,58.0,43.0,TRAPPIST-1e,A,Europa,49.0,6715.0,0003_01,False
3,False,AS,33.0,0.0,TRAPPIST-1e,A,Europa,193.0,3329.0,0003_02,False
4,False,FS,16.0,303.0,TRAPPIST-1e,F,Earth,2.0,565.0,0004_01,True


In [24]:
# Define the preprocessor

from sklearn.compose import ColumnTransformer

spending_features = ['RoomService', 'Spa', 'VRDeck'] # bin then OHE
categorical_features = ['HomePlanet', 'CryoSleep', 'Destination', 'Deck', 'DeckSide'] # OHE

preprocessor = ColumnTransformer(
    transformers=[
        ('category', categorical_pipeline, categorical_features),
        ('spending', spending_pipeline, spending_features),
        ('age', age_pipeline, ['Age'])
    ],
    remainder='passthrough'  # This leaves the rest of the columns untouched
)

In [25]:
y = train_df.pop('Transported')

In [26]:
i = train_df.pop('PassengerId')

In [27]:
# Now you can fit and transform the training data with the preprocessor
X = preprocessor.fit_transform(train_df)
# if sparse convert to dense array
from scipy.sparse import csr_matrix
if type(X) == csr_matrix:
    X = X.toarray()

## Feature selection

We can feed the results back into the "Drop Unrequired columns" section

In [28]:
from sklearn.feature_selection import f_regression

# assuming X is your feature matrix and y is the target variable
F, p = f_regression(X, y)

# print the F-values for each feature
f_corr = pd.DataFrame({'Feature': list(preprocessor.get_feature_names_out()), 'F-value': F}, columns=['Feature', 'F-value'])

f_corr.sort_values(by = 'F-value', ascending = False)

,Feature,F-value
4,category__CryoSleep_False,2427.125458
5,category__CryoSleep_True,2334.295199
37,spending__RoomService,536.491728
38,spending__Spa,435.917591
39,spending__VRDeck,380.771546
1,category__HomePlanet_Europa,280.812746
0,category__HomePlanet_Earth,255.580886
12,category__Deck_B,185.951263
23,category__DeckSide_BS,143.189246
40,age__Age_0.0,138.847979


In [29]:
# So if we set the cutoff as an F Value of 100 we will select the following features
selected_features_raw = f_corr[f_corr['F-value'] > 100]['Feature'].values

In [30]:
def feature_extractor(rawf: str):
    if rawf == None:
        return None
    elif '__' not in rawf:
        return rawf
    
    temp = rawf.split('__')[1]
    return temp.split('_')[0]
selected_features = [feature_extractor(f) for f in selected_features_raw]
selected_features

['HomePlanet',
 'HomePlanet',
 'CryoSleep',
 'CryoSleep',
 'Destination',
 'Deck',
 'Deck',
 'DeckSide',
 'DeckSide',
 'RoomService',
 'Spa',
 'VRDeck',
 'Age']

In [31]:
unique_features = list(set(selected_features))
unique_features

['Deck',
 'Destination',
 'CryoSleep',
 'VRDeck',
 'DeckSide',
 'HomePlanet',
 'Age',
 'Spa',
 'RoomService']

# Time to attempt the modelling

## Split the data to create a validation set

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.25, random_state = 0)

## Instantiation of the models

In [33]:
classifiers = {
    'LogisticRegression': LogisticRegression(),
    'Perceptron': Perceptron(),
    'SVC': SVC(),
    'KNN': KNeighborsClassifier(n_neighbors = 5),
    'DecisionTree': DecisionTreeClassifier(),
    'Gaussian': GaussianNB(),
    'RandomForest': RandomForestClassifier(),
    'LinearSvc': LinearSVC(),
    'SGDClassifier': SGDClassifier(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

## Train the models

### Evaluate model

In [34]:
def score_model(classifier, X_train, X_val, y_train, y_val):
    # Fit the model
    classifier.fit(X_train, y_train)

    # Predict the test data
    y_pred = classifier.predict(X_val)

    # Create a confusion matrix
    cm = confusion_matrix(y_val, y_pred)

    # Calculate the accuracy
    acc = accuracy_score(y_val, y_pred)

    scores = cross_val_score(classifier, X_train, y_train, cv = 10)

    return { 
            'confusion_matrix': cm,
            'accuracy': acc,
            'kfold-cv': scores.mean()
        }

def run_classifiers(classifiers: dict, X_train, X_test, y_train, y_val) -> dict:
    results = {}
    for name, classifier in classifiers.items():
        results[name] = score_model(classifier, X_train, X_test, y_train, y_val)
    return results

results = run_classifiers(classifiers, X_train, X_val, y_train, y_val)

In [35]:
# convert results to a dataframe
results_df = pd.DataFrame(results).T
results_df.sort_values(by='kfold-cv', ascending=False)

,confusion_matrix,accuracy,kfold-cv
LogisticRegression,"[[817, 260], [200, 897]]",0.788408,0.784323
XGBoost,"[[817, 260], [206, 891]]",0.785649,0.783097
LinearSvc,"[[807, 270], [184, 913]]",0.791168,0.779108
RandomForest,"[[851, 226], [233, 864]]",0.788868,0.7765
SVC,"[[734, 343], [170, 927]]",0.764029,0.774964
KNN,"[[853, 224], [286, 811]]",0.765409,0.75886
DecisionTree,"[[737, 340], [251, 846]]",0.728151,0.738767
Perceptron,"[[871, 206], [315, 782]]",0.76035,0.736929
SGDClassifier,"[[801, 276], [172, 925]]",0.793928,0.736143
Gaussian,"[[523, 554], [113, 984]]",0.693192,0.692437


## Tuning the two best models

In [36]:
# Print the confusion matrix with labels for TP, FP, TN, FN
def print_confusion_matrix(cm: np.ndarray):
    # True Positives
    TP = cm[1, 1]
    # True Negatives
    TN = cm[0, 0]
    # False Positives
    FP = cm[0, 1]
    # False Negatives
    FN = cm[1, 0]

    # Now we have extracted the values print as a grid
    print(f'True Positives: {TP} False Positives: {FP}')
    print(f'False Negatives: {FN} True Negatives: {TN}')
    

## Tuning the two best models

In [37]:
# Tune a model using GridSearchCV
def tune_model(classifier, param_grid):
    print(f'\n\nTuning {classifier.__class__.__name__}...')
    grid_search = GridSearchCV(classifier, param_grid, cv=10, n_jobs = -1, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    best_estimator = grid_search.best_estimator_
    score = score_model(best_estimator, X_train, X_val, y_train, y_val)

    print(f'Best SVC Accuracy: { score["accuracy"] }')
    print(f'Best SVC KFold CV: { score["kfold-cv"] }')
    print('\nConfusion Matrix:')
    print_confusion_matrix(score['confusion_matrix'])
    
    f1 = f1_score(y_val, best_estimator.predict(X_val))
    # and finally the f1 score
    print(f'F1 Score: {f1}')

    return (grid_search.best_params_, best_estimator, f1)

# Print the confusion matrix with labels for TP, FP, TN, FN
def print_confusion_matrix(cm: np.ndarray):
    # True Positives
    TP = cm[1, 1]
    # True Negatives
    TN = cm[0, 0]
    # False Positives
    FP = cm[0, 1]
    # False Negatives
    FN = cm[1, 0]

    # Now we have extracted the values print as a grid
    print(f'True Positives: {TP} False Positives: {FP}')
    print(f'False Negatives: {FN} True Negatives: {TN}')

In [38]:
tuning_results = [
    tune_model(
        XGBClassifier(),
        param_grid = {
            'n_estimators': [100, 200, 300],
            'max_depth': [3, 4, 5],
            'learning_rate': [0.1, 0.01, 0.001],
            'eval_metric': ['logloss'], 
            'use_label_encoder': [False]
        }, 
    ),
    tune_model(
        RandomForestClassifier(), 
        param_grid = {
            'n_estimators': [100, 200, 300],
            'max_depth': [3, 4, 5],
            'criterion': ['gini', 'entropy']
        }
    )
]



Tuning XGBClassifier...
Best SVC Accuracy: 0.795768169273229
Best SVC KFold CV: 0.7935276544815432

Confusion Matrix:
True Positives: 928 False Positives: 275
False Negatives: 169 True Negatives: 802
F1 Score: 0.8069565217391304


Tuning RandomForestClassifier...
Best SVC Accuracy: 0.7442502299908004
Best SVC KFold CV: 0.7487355460688134

Confusion Matrix:
True Positives: 724 False Positives: 183
False Negatives: 373 True Negatives: 894
F1 Score: 0.722554890219561


In [39]:
best_result = max(tuning_results, key=lambda x: x[2])[1]

## Create submission

Now we can predict the test set and submit

In [40]:
# Convert any Boolean columns with Unknown to String type
for column in test_df.columns:
    if 'Unknown' in test_df[column].unique():
        print(column)
        test_df[column] = test_df[column].astype(str)

CryoSleep
DeckSide
Destination
Deck
HomePlanet


In [41]:
# Check the shape

print(f'Test: {test_df.shape}')

# Print the first 5 rows:
test_df.head()

Test: (4277, 10)


,CryoSleep,DeckSide,Age,RoomService,Destination,Deck,HomePlanet,VRDeck,Spa,PassengerId
0,True,GS,27.0,0.0,TRAPPIST-1e,G,Earth,0.0,0.0,0013_01
1,False,FS,19.0,0.0,TRAPPIST-1e,F,Earth,0.0,2823.0,0018_01
2,True,CS,31.0,0.0,55 Cancri e,C,Europa,0.0,0.0,0019_01
3,False,CS,38.0,0.0,TRAPPIST-1e,C,Europa,585.0,181.0,0021_01
4,False,FS,20.0,10.0,TRAPPIST-1e,F,Earth,0.0,0.0,0023_01


In [42]:
# Capture the passenger IDs for submission and remove from the data as not useful predictors
idx_test = test_df.pop('PassengerId')

In [43]:
# Pre-process test set
X_test = preprocessor.transform(test_df) # sparse to dense array

In [44]:
# Predict the test set
y_pred = best_result.predict(X_test).astype(bool)

In [46]:
# Write the submission DataFrame
submission = pd.DataFrame({'PassengerId': idx_test, 'Transported': y_pred})
submission.to_csv(f'submissions/{model}-submission.csv', index=False)
print("Your submission was successfully saved!")


Your submission was successfully saved!
